In [1]:
import gym
import random
import numpy as np
import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression
from statistics import mean, median
from collections import Counter

W1209 23:27:43.767674 4738356672 deprecation_wrapper.py:119] From /Users/abhay.shukla/anaconda3/lib/python3.7/site-packages/tflearn/helpers/summarizer.py:9: The name tf.summary.merge is deprecated. Please use tf.compat.v1.summary.merge instead.

W1209 23:27:43.768696 4738356672 deprecation_wrapper.py:119] From /Users/abhay.shukla/anaconda3/lib/python3.7/site-packages/tflearn/helpers/trainer.py:25: The name tf.summary.FileWriter is deprecated. Please use tf.compat.v1.summary.FileWriter instead.

W1209 23:27:43.773744 4738356672 deprecation_wrapper.py:119] From /Users/abhay.shukla/anaconda3/lib/python3.7/site-packages/tflearn/collections.py:13: The name tf.GraphKeys is deprecated. Please use tf.compat.v1.GraphKeys instead.

W1209 23:27:43.776864 4738356672 deprecation_wrapper.py:119] From /Users/abhay.shukla/anaconda3/lib/python3.7/site-packages/tflearn/config.py:123: The name tf.get_collection is deprecated. Please use tf.compat.v1.get_collection instead.

W1209 23:27:43.782298 47383566

In [2]:
LR = 1e-3

In [3]:
env = gym.make('CartPole-v0')

In [4]:
env.reset()

array([ 0.03743796, -0.04633037,  0.01107057, -0.04054717])

In [5]:
goal_steps = 500

In [6]:
score_requirement = 50

In [7]:
initial_games = 10000

In [8]:
def some_random_games_first():
    for episode in range(5):
        env.reset()
        for t in range(goal_steps):
            env.render()
            action = env.action_space.sample()
            observation, reward, done, info = env.step(action)
            if done:
                break

In [9]:
# some_random_games_first()

In [10]:
def initial_population():
    # [OBS, MOVES]
    training_data = []
    # all scores:
    scores = []
    # just the scores that met our threshold:
    accepted_scores = []
    # iterate through however many games we want:
    for _ in range(initial_games):
        score = 0
        # moves specifically from this environment:
        game_memory = []
        # previous observation that we saw
        prev_observation = []
        # for each frame in 200
        for _ in range(goal_steps):
            # choose random action (0 or 1)
            action = random.randrange(0,2)
            # do it!
            observation, reward, done, info = env.step(action)
            
            # notice that the observation is returned FROM the action
            # so we'll store the previous observation here, pairing
            # the prev observation to the action we'll take.
            if len(prev_observation) > 0 :
                game_memory.append([prev_observation, action])
            prev_observation = observation
            score+=reward
            if done: break

        # IF our score is higher than our threshold, we'd like to save
        # every move we made
        # NOTE the reinforcement methodology here. 
        # all we're doing is reinforcing the score, we're not trying 
        # to influence the machine in any way as to HOW that score is 
        # reached.
        if score >= score_requirement:
            accepted_scores.append(score)
            for data in game_memory:
                # convert to one-hot (this is the output layer for our neural network)
                if data[1] == 1:
                    output = [0,1]
                elif data[1] == 0:
                    output = [1,0]
                    
                # saving our training data
                training_data.append([data[0], output])

        # reset env to play again
        env.reset()
        # save overall scores
        scores.append(score)
    
    # just in case you wanted to reference later
    training_data_save = np.array(training_data)
    np.save('saved.npy',training_data_save)
    
    # some stats here, to further illustrate the neural network magic!
    print('Average accepted score:',mean(accepted_scores))
    print('Median score for accepted scores:',median(accepted_scores))
    print(Counter(accepted_scores))
    
    return training_data

In [11]:
def neural_network_model(input_size):

    network = input_data(shape=[None, input_size, 1], name='input')

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 512, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 256, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 128, activation='relu')
    network = dropout(network, 0.8)

    network = fully_connected(network, 2, activation='softmax')
    network = regression(network, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')
    model = tflearn.DNN(network, tensorboard_dir='log')

    return model


def train_model(training_data, model=False):

    X = np.array([i[0] for i in training_data], dtype=np.float64).reshape(-1,len(training_data[0][0]),1)
    y = np.array([i[1] for i in training_data], dtype=np.float64)

    if not model:
        model = neural_network_model(input_size = len(X[0]))
    
    model.fit({'input':X}, {'targets':y}, n_epoch=3, snapshot_step=500, show_metric=True, run_id='openai_learning')
    return model

In [12]:
training_data = initial_population()

Average accepted score: 60.53083109919571
Median score for accepted scores: 57.0
Counter({51.0: 33, 53.0: 29, 54.0: 27, 56.0: 24, 50.0: 21, 58.0: 21, 55.0: 21, 52.0: 18, 57.0: 15, 63.0: 15, 60.0: 14, 61.0: 11, 68.0: 10, 59.0: 10, 67.0: 9, 62.0: 9, 65.0: 9, 70.0: 8, 64.0: 7, 71.0: 7, 73.0: 7, 69.0: 6, 66.0: 4, 74.0: 4, 72.0: 4, 75.0: 4, 76.0: 4, 77.0: 3, 85.0: 3, 87.0: 2, 84.0: 2, 79.0: 1, 96.0: 1, 119.0: 1, 82.0: 1, 93.0: 1, 107.0: 1, 89.0: 1, 108.0: 1, 102.0: 1, 88.0: 1, 100.0: 1, 97.0: 1})


In [13]:
model = train_model(training_data)

Training Step: 1040  | total loss: 0.67345 | time: 4.347s
| Adam | epoch: 003 | loss: 0.67345 - acc: 0.5969 -- iter: 22144/22205
Training Step: 1041  | total loss: 0.66760 | time: 4.359s
| Adam | epoch: 003 | loss: 0.66760 - acc: 0.6060 -- iter: 22205/22205
--


In [14]:
model.save('cartpole_model_v0.model')

In [15]:
scores = []
choices = []

for each_game in range(10):
    score = 0
    game_memory = []
    prev_obs = []
    env.reset()
    for _ in range(goal_steps):
        env.render()
        if len(prev_obs) == 0:
            action = random.randrange(0,2)
        else:
            action = np.argmax(model.predict(prev_obs.reshape(-1, len(prev_obs), 1))[0])
        choices.append(action)
        new_observation, reward, done, info = env.step(action)
        prev_obs = new_observation
        game_memory.append([new_observation, action])
        score += reward
        if done:
            break
    scores.append(score)
    
print('Average score:', sum(scores)/len(scores))
print('Choice 1: {}, Choice 2: {}'.format(choices.count(1)/len(choices), choices.count(0)/len(choices)))

Average score: 200.0
Choice 1: 0.5, Choice 2: 0.5
